In [1]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
import ast

# Training and Testing Multinomial Naive Bayes Classifier

## Reading in Training Data and Simple Transforms

In [2]:
train = pd.read_csv('./data/processed/goodreads_books_train_processed.csv',
                    converters={'cleaned_descriptions':ast.literal_eval},
                    index_col=0)
train.head()

,is_english_description,cleaned_descriptions,genre_1,votes_1
index,,,,
0,True,"[filmmak, chase, ryan, keith, ellison, complet...",Fiction,174.0
2,True,"[incred, nation, bestsel, chang, peopl, live, ...",Self Help,1162.0
3,True,"[high, spirit, intellig, good, look, abigail, ...",Romance,550.0
4,True,"[news, went, sylvest, rayn, eleg, impecc, duke...",Romance,725.0
5,True,"[major, motion, pictur, star, nicola, cage, di...",Gothic,106.0


Scikit-learn has the ability to do all of the text cleaning we have done already EXCEPT stemming, so I'll use our cleaned descriptions after stitching them back together such that each description is a string.

We are passing counts of unique words as features, meaning that the dimensions of `X_train` are `(number of descriptions, number of unique words)`.  The targets in `y_train` are just strings representing each genre.

In [3]:
train['mod_descriptions'] = train['cleaned_descriptions'].apply(lambda x: ' '.join(x))

valid_train = train[train[['mod_descriptions', 'genre_1']].notnull().all(axis=1)]

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(valid_train.mod_descriptions)
y_train = valid_train['genre_1']

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

## Read in Test Data

In [4]:
val = pd.read_csv('./data/processed/goodreads_books_val_processed.csv',
                   converters={'cleaned_descriptions':ast.literal_eval},
                   index_col=0)
val.head()

,is_english_description,cleaned_descriptions,genre_1,votes_1
index,,,,
0,True,"[1968, lynni, young, white, woman, development...",Romance,467.0
1,True,"[miranda, wake, alon, park, bench, memori, pan...",Young Adult,276.0
2,True,"[âmotherâ, dayâ, part, one, talia, al, g...",Superheroes,13.0
3,True,"[asin, b00dh4ssy6, ha, move, latest, edit, onc...",NaN,NaN
4,True,"[ladi, evelynâ, life, chang, mother, die, st...",NaN,NaN


## Make Similar Transforms and Score

In [5]:
val['mod_descriptions'] = val['cleaned_descriptions'].apply(lambda x: ' '.join(x))
valid_val = val[val[['mod_descriptions', 'genre_1']].notnull().all(axis=1)]
X_val = vectorizer.transform(valid_val.mod_descriptions)
y_val = valid_val[['genre_1']]

In [6]:
classifier.score(X_val,y_val)

0.4371867979955072

The score above is actually not bad considering how overlapping the genres are... It could probably be greatly improved just by making genres more distinct.